In [40]:
from selenium.webdriver.chrome.options import Options
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from flask import Flask, render_template

In [121]:
import os
import re
import requests
from flask import Flask, render_template, request, send_from_directory
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [128]:
app = Flask(__name__)

# 이미지 저장 경로 설정
IMAGE_FOLDER = 'static/images'
app.config['IMAGE_FOLDER'] = IMAGE_FOLDER

# 특수 문자 제거 함수
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "", filename)

# 이미지 다운로드 함수
def download_image(image_url, image_name):
    image_name = sanitize_filename(image_name)  # 파일 이름에서 특수 문자 제거
    image_path = os.path.join(IMAGE_FOLDER, image_name)
    if not os.path.exists(IMAGE_FOLDER):
        os.makedirs(IMAGE_FOLDER)  # 경로가 없으면 생성
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(image_path, 'wb') as f:
            f.write(response.content)
    return image_name

# 크롤링 함수 정의 (검색어를 매개변수로 받음)
def crawl_contests(search_keyword):
    driver = webdriver.Chrome()  # ChromeDriver 경로 설정 필요 시 추가
    url = 'https://www.wevity.com/?c=find&s=1&gub=&cidx=&sp=&sw=&gbn=list&mode=ing'
    driver.get(url)
    driver.implicitly_wait(10)
    time.sleep(3)

    # 페이지 스크롤
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    contests = []
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="sw"]'))
        )
        search_boxes = driver.find_elements(By.CSS_SELECTOR, 'input[name="sw"]')
        search_box = search_boxes[1]
        search_box.clear()
        search_box.send_keys(search_keyword)

        search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value="검색"]')
        search_button.click()

        time.sleep(3)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.list'))
        )

        contest_list = driver.find_elements(By.CSS_SELECTOR, 'ul.list li:not(.top)')

        for i in range(len(contest_list)):
            contest_list = driver.find_elements(By.CSS_SELECTOR, 'ul.list li:not(.top)')  # 다시 요소를 찾음
            contest = contest_list[i]

            title = contest.find_element(By.CSS_SELECTOR, '.tit a').text
            link = contest.find_element(By.CSS_SELECTOR, '.tit a').get_attribute('href')
            category = contest.find_element(By.CSS_SELECTOR, '.sub-tit').text
            host = contest.find_element(By.CSS_SELECTOR, '.organ').text
            days_left = contest.find_element(By.CSS_SELECTOR, '.day').text.strip()
            views = contest.find_element(By.CSS_SELECTOR, '.read').text

            # 상세 페이지로 이동
            driver.get(link)
            time.sleep(3)

            # 이미지 URL 추출
            try:
                image_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#viewContents img'))
                )
                image_url = image_element.get_attribute('src')
                image_name = title.replace(" ", "_") + '.jpg'  # 이미지 파일 이름

                # 이미지 다운로드
                download_image(image_url, image_name)
            except Exception as e:
                print(f"이미지를 찾을 수 없음: {e}")
                image_name = None

            # 내용 추출
            try:
                description_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.comm-desc'))
                )
                description_html = description_element.get_attribute('innerHTML')
                description = description_html.replace("\n", "<br>").replace("<div>", "<p>").replace("</div>", "</p>")
            except Exception as e:
                print(f"설명 정보를 찾을 수 없음: {e}")
                description = "설명 정보가 없습니다."

            contests.append({
                'title': title,
                'link': link,
                'category': category,
                'host': host,
                'days_left': days_left,
                'views': views,
                'image': image_name,  # 다운로드한 이미지 파일 이름
                'description': description  # 추가된 설명
            })

            # 목록 페이지로 돌아가기
            driver.back()
            time.sleep(3)

            # 목록 페이지로 돌아온 후, 다시 contest_list 요소를 찾음
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.list'))
            )

    except Exception as e:
        print(f"오류 발생: {e}")

    finally:
        driver.quit()

    return contests

In [129]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/contests', methods=['GET'])
def contests():
    search_keyword = request.args.get('search_keyword')  # GET 요청으로 전달된 검색어 받기
    if search_keyword:
        contests_info = crawl_contests(search_keyword)  # 크롤링된 공모전 정보를 가져옴
        return render_template('contests.html', contests=contests_info)
    else:
        return render_template('contests.html', contests=[])

# 로컬에 저장된 이미지를 제공하는 라우트
@app.route('/image/<filename>')
def image(filename):
    return send_from_directory(app.config['IMAGE_FOLDER'], filename)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2024 09:38:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 09:38:42] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /contests?search_keyword=sw HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /static/images/2024년_ICT/SW_여성_창업_공모전_SPECIAL.jpg HTTP/1.1" 404 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /upload/contest/20240716223817_3030a9df.jpg HTTP/1.1" 404 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /upload/contest/20240902160524_38f35789.jpg HTTP/1.1" 404 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /static/images/2024_전국_청소년_오픈SW_GAME_코딩대회_with파이썬.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 09:39:36] "GET /static/images/2024년_ICT/SW_여성_창업_공모전.jpg HTTP/1.1" 404 -
